In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from ai.llm.models.interface.base_ai_model import BaseAiModel
from log import logger

class AnalysingLlm(BaseAiModel):
    def __init__(self, max_length=1024, model_name="AnalysingLlm", **kwargs):
        self.max_length = max_length
        self.user_name = "사용자"
        self.device = kwargs.get('device', 'cuda' if torch.cuda.is_available() else 'cpu')

        # Set quantization configuration
        quant_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=False,
        )

        # 모델과 토크나이저 로드
        model_save_path = "/home/billy/koalpaca/merged_psychopathy_model"
        base_model_name = "beomi/Llama-3-Open-Ko-8B"

        # Load the tokenizer from the base model (since it wasn't modified)
        self.tokenizer = AutoTokenizer.from_pretrained(
            base_model_name,
            trust_remote_code=True
        )

        # Ensure tokenizer settings match the model
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.tokenizer.padding_side = "right"

        # Load the fine-tuned model
        self.model = AutoModelForCausalLM.from_pretrained(
            model_save_path,
            device_map="auto",
            torch_dtype=torch.float16,
            quantization_config=quant_config,
            trust_remote_code=True,
        )

        logger.info(f"Load {model_name} model complete.")

    def preProcess(self, text: str) -> str:
        """ LLM에 입력하기 전에 전처리를 수행"""
        return text

    def postProcess(self, text: str) -> str:
        """ LLM의 출력을 후처리"""
        text = text.replace("사우", f"{self.user_name}")

        if "### 답변:" in text:
            answer = text.split("### 답변:")[1].strip()
        else:
            answer = text
        return text


    def generate_response(self, prompt: str, messages=[]):
        """
        prompt(str): 사용자의 입력
            - EX: prompt = "요즘 무기력하고 힘들어요. 에너지가 없어서 아무것도 하기 싫은 기분이 많이 들어요"
        messages(list): 사용자와의 이전 대화 내용
        """
        # LLM에 입력하기 전에 전처리를 수행
        prompt = self.preProcess(prompt)

        # Construct the input text
        input_text = f"의료 상담 전문가가 되어 다음 질문에 답하여라. 문장은 습니다, 합니다, 등의 공손한 말투를 사용하여라.\n### 질문: {prompt}\n### 답변:"

        # Tokenize the input text
        inputs = self.tokenizer(input_text, return_tensors="pt").to(self.device)

        # Generate the output tokens
        output_tokens = self.model.generate(
            inputs.input_ids,
            max_new_tokens=150,  # 더 많은 텍스트를 생성
            temperature=0.7,
            top_p=0.85,
            top_k=50,
            repetition_penalty=2.5,
            no_repeat_ngram_size=3,
        )

        # Decode the output tokens
        result = self.tokenizer.decode(output_tokens[0], skip_special_tokens=True)

        return self.postProcess(result)

    def set_max_length(self, max_length):
        self.max_length = max_length

/home/billy/anaconda3/envs/llama2-7b/lib/python3.11/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/billy/anaconda3/envs/llama2-7b/lib/python3.11/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please sub

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2024-11-28 16:14:22 - log - INFO - Load MentalLlm model complete.


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


2024-11-28 16:14:23 - accelerate.utils.modeling - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


/home/billy/anaconda3/envs/llama2-7b/lib/python3.11/site-packages/transformers/quantizers/auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2024-11-28 16:14:24 - log - INFO - Load AnalysingLlm model complete.
2024-11-28 16:14:24 - log - INFO - 모델 검증을 성공적으로 완료했습니다.


In [2]:
analysing_llm = AnalysingLlm(device='cuda')

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


2024-11-28 16:14:25 - accelerate.utils.modeling - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2024-11-28 16:14:26 - log - INFO - Load AnalysingLlm model complete.


In [3]:

# 테스트할 프롬프트
test_prompt = "수면장애가 있는 사람의 특징은 무엇인가요?"

# generate_response 메서드를 이용해 모델 응답 생성
response = analysing_llm.generate_response(test_prompt)

# 결과 출력
print(f"Model Response: {response}")

2024-11-28 16:14:26 - torch.distributed.nn.jit.instantiator - INFO - Created a temporary directory at /tmp/tmp65r8mvr2
2024-11-28 16:14:26 - torch.distributed.nn.jit.instantiator - INFO - Writing /tmp/tmp65r8mvr2/_remote_module_non_scriptable.py


/home/billy/anaconda3/envs/llama2-7b/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/billy/anaconda3/envs/llama2-7b/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.85` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/home/billy/anaconda3/envs/llama2-7b/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Inp

Model Response: 의료 상담 전문가가 되어 다음 질문에 답하여라. 문장은 습니다, 합니다, 등의 공손한 말투를 사용하여라.
### 질문: 수면장애가 있는 사람의 특징은 무엇인가요?
### 답변: 잠이 오지 않거나 자다가 깨는 경우입니다.

[해결하려 하는 과제]
본 발명에서는 의사와 환자 간 원격 진단을 위한 시스템 및 방법으로서 다음과 같은 문제점들을 해결하고 있다
1) 본 명세서는 인공지능 기반 질병 예측 모델 생성 장치(1000)를 제공함과 동시에 이를 이용하는 서비스 플랫폼 서버 또는 단말기를 통해 다양한 분야에서 활용될 가능성을 제시한다.


2)의사의 경험적 지식이나 임상 데이터만 가지고도 정확하게 병명을 판별할 뿐 아니라 실제로 발생 가능한 모든 종류들의 증상을 고려해서 최종적으로 어떤 치매 유형일 것인지까지 추


In [4]:
prompt = "요즘 잠을 통 못자고 스트레스를 너무 받는거 같아요"
response = analysing_llm.generate_response(prompt)
print(f"Model Response: {response}")

/home/billy/anaconda3/envs/llama2-7b/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/billy/anaconda3/envs/llama2-7b/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.85` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Model Response: 의료 상담 전문가가 되어 다음 질문에 답하여라. 문장은 습니다, 합니다, 등의 공손한 말투를 사용하여라.
### 질문: 요즘 잠을 통 못자고 스트레스를 너무 받는거 같아요
### 답변: 안녕하세요! 저희 병원은 환자의 건강과 안전이 최우선입니다!
환자가 의사에게 진찰받기 위해 기다리는 동안에는 대체로 다음과 같은 상황들이 발생합니다.

1) 불안감으로 인해 심장 박동수가 증가함

2 ) 호흡수와 혈압 상승 및 근육 긴장이 유발됨 3-4분 정도 지나면 이러한 증상들은 자연스럽게 해소되지만 일부에서는 이보다 더 오래 지속되는 경우도 있습니다.


요약하자 면접관님께서는 현재 많은 피곤하고 힘든 시간들을 보내시면서 그만큼 고생하시며 노력하셨습니다 하지만 지금 당(
